In [1]:
from QuditsOnQubits import create_ame43
from qiskit import qpy
from qiskit.circuit import QuantumCircuit

In [2]:
ame43 = create_ame43()

In [3]:
with open('Zgate.qpy', 'rb') as fd:
    Zgate = qpy.load(fd)[0]

with open('Xgate.qpy', 'rb') as fd:
    Xgate = qpy.load(fd)[0]

In [4]:
operators = []

In [5]:
G1 = QuantumCircuit(8)
G1.append(Xgate, [0, 1])
G1.append(Zgate, [2, 3])
G1.append(Zgate, [6, 7])
operators.append(G1)

In [6]:
G1G2 = QuantumCircuit(8)
G1G2.append(Xgate, [0, 1])
G1G2.append(Zgate, [0, 1])
G1G2.append(Zgate, [2, 3])
G1G2.append(Xgate, [2, 3])
G1G2.append(Zgate, [4, 5])
G1G2.append(Zgate, [6, 7])
operators.append(G1G2)

In [7]:
G1G22 = QuantumCircuit(8)
G1G22.append(Xgate, [0, 1])
G1G22.append(Zgate, [0, 1])
G1G22.append(Zgate, [0, 1])
G1G22.append(Zgate, [2, 3])
G1G22.append(Xgate, [2, 3])
G1G22.append(Xgate, [2, 3])
G1G22.append(Zgate, [4, 5])
G1G22.append(Zgate, [4, 5])
G1G22.append(Zgate, [6, 7])
operators.append(G1G22)

In [8]:
G3 = QuantumCircuit(8)
G3.append(Zgate, [2, 3])
G3.append(Xgate, [4, 5])
G3.append(Zgate, [6, 7])
G3.append(Zgate, [6, 7])
operators.append(G3)

In [9]:
G1G4 = QuantumCircuit(8)
G1G4.append(Xgate, [0, 1])
G1G4.append(Zgate, [0, 1])
G1G4.append(Zgate, [2, 3])
G1G4.append(Zgate, [4, 5])
G1G4.append(Zgate, [4, 5])
G1G4.append(Zgate, [6, 7])
G1G4.append(Xgate, [6, 7])
operators.append(G1G4)

In [10]:
final_circs = []

for i in operators:
    tempcirc = ame43.copy()
    tempcirc.append(i, [0, 1, 2, 3, 4, 5, 6, 7])
    tempcirc.measure_all()
    final_circs.append(tempcirc)

In [11]:
import numpy as np

def exp_bell(counts, shots, k):
    oper = 0
    keys = list(counts.keys())[k]
    for i in list(counts.keys()):
        p = counts[i]/shots
        sum = 0
        for j in range(0, len(i), 2):
            if f'{keys[j]}{keys[j+1]}' == "00":
                sum += 0
            elif f'{keys[j]}{keys[j+1]}' == "01":
                sum += 1
            elif f'{keys[j]}{keys[j+1]}' == "10":
                sum += 2
        omega = np.exp(2*np.pi*1.j/3)
        f = pow(omega, sum)
        oper += p*f

    return oper

In [12]:
from qiskit.primitives import StatevectorSampler

bell_exp_list = []
exps_operators = []

shots = 10000
sampler = StatevectorSampler()

for i, j in zip(final_circs, range(len(final_circs))):
    job = sampler.run([i], shots = shots)
    data_pub = job.result()[0].data
    counts = data_pub.meas.get_counts()
    #exps_operators.append(str_list_gates[j])
    bell_exp = exp_bell(counts, shots, j)
    bell_exp_list.append(np.absolute(bell_exp))

In [13]:
bell_exp_list

[np.float64(1.0),
 np.float64(0.9999999999999998),
 np.float64(0.9999999999999999),
 np.float64(0.9999999999999999),
 np.float64(1.0)]

In [14]:
2 * (bell_exp_list[0] + bell_exp_list[1] + 0.5 * (bell_exp_list[2] + bell_exp_list[3]) + bell_exp_list[4])

np.float64(7.999999999999999)

In [15]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

service = QiskitRuntimeService(channel="ibm_quantum", token="69c33c504eae6077ab5a35bc23ed37d0e2ce46a8d88b9c3f72fd32883335a986ef5c86d80f8b561cbdbe773a8d56475d710fd741d28597d1148e2dcf74fe488d")

In [16]:
real_backend = service.backend("ibm_brisbane")
aer = AerSimulator.from_backend(real_backend)

In [17]:
pm = generate_preset_pass_manager(backend=aer, optimization_level=1)
sampler = Sampler(mode=aer)

bell_exp_list_sim = []
exps_operators_sim = []

shots_sim = 1000

for k, l in zip(final_circs, range(len(final_circs))):
    isa_qc = pm.run(k)

    jobSim = sampler.run([isa_qc], shots = shots_sim)
    data_pubSim = jobSim.result()[0].data
    countsSim = data_pubSim.meas.get_counts()
    #exps_operators.append(str_list_gates[j])
    bell_exp_sim = exp_bell(countsSim, shots_sim, l)
    bell_exp_list_sim.append(np.absolute(bell_exp_sim))

In [18]:
bell_exp_list_sim

[np.float64(1.0000000000000007),
 np.float64(1.000000000000002),
 np.float64(1.0000000000000016),
 np.float64(1.0000000000000004),
 np.float64(1.0000000000000018)]

In [45]:
backend = service.least_busy(simulator=False, operational=True)

In [46]:
backend

<IBMBackend('ibm_torino')>

In [51]:
backend = service.least_busy(simulator=False, operational=True)
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc_real = pm.run(final_circs[0])
jobSim = sampler.run([isa_qc_real], shots = shots_sim)
data_pubSim = jobSim.result()

AerError: 'unknown instruction: cz'

In [22]:
data_pubSim = jobSim.result()[0].data
countsSim = data_pubSim.meas.get_counts()
#exps_operators.append(str_list_gates[j])
bell_exp_sim = exp_bell(countsSim, shots_sim, 0)
bell_exp_list_sim.append(np.absolute(bell_exp_sim))

AerError: 'unknown instruction: cz'

In [49]:
for m, n in zip(final_circs, range(len(final_circs))):
    isa_qc = pm.run(m)

    jobSim = sampler.run([isa_qc], shots = shots_sim)
    data_pubSim = jobSim.result()[0].data
    countsSim = data_pubSim.meas.get_counts()
    #exps_operators.append(str_list_gates[j])
    bell_exp_sim = exp_bell(countsSim, shots_sim, n)
    bell_exp_list_sim.append(np.absolute(bell_exp_sim))

In [50]:
bell_exp_list_sim

[np.float64(1.000000000000002),
 np.float64(1.0000000000000016),
 np.float64(1.0000000000000007),
 np.float64(1.000000000000002),
 np.float64(1.000000000000002),
 np.float64(1.0000000000000018),
 np.float64(1.0000000000000024),
 np.float64(1.0000000000000013),
 np.float64(1.0000000000000007),
 np.float64(1.0000000000000022)]

In [83]:
obs_circs = [G1, G1G2, G1G22, G3, G1G4]